## Transfer Learning Tuning:
  * model parameter and other information can be found below:
     * [source](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
     
  * base_model: use vgg16 and freeze at bottleneck layer (stop right before flatten layer) 
  * top_model: tune dense layers (parameters are inspired by source)
  
---
#### This cell is required in order to use GPU for running the model 

In [3]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import keras
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras
keras.backend.get_session().run(tf.global_variables_initializer())  

---
#### Import necessary libraries

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
from keras import optimizers

---
#### Import train_df and test_df

In [6]:
train_df = pd.read_pickle('../pickle_files/train_df_glasses.pkl')
test_df = pd.read_pickle('../pickle_files/test_df_glasses.pkl')

In [8]:
train_df.shape[0]

505

#### Get bottleneck features to tune top models

In [9]:
def save_bottleneck_features(train_df, test_df, label, batch_size):
    '''
    inputs:
    train_df, test_df: train and test dataframes saved in pickle_files folder
    label: a string, eyewear, hat, or beard
    batch_size: process images in batches
    outputs:
    saves bottleneck features inside folder tuning_data as npy file
    '''
    # intialize the vgg16 model 
    # make sure not to train the top layers 
    base_model = VGG16(weights = 'imagenet', include_top = False)
    # create train_generator and test_generator to get bottleneck inputs for train and test df 
    # no image preprocessing other than rescale 
    datagen = ImageDataGenerator(rescale=1. / 255)
    # make sure shuffle is False so we know the label follows the sequence of the dataframe 
    # so we can tune top_model 
    train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    shuffle = False,
    target_size=(150,150),
    class_mode = None)
    # get features saved as .npy in tunign_data folder 
    bottleneck_features_train = base_model.predict_generator(
        train_generator, train_df.shape[0]//batch_size)
    np.save(open('../tuning_data/bottleneck_features_train.npy','wb'),
           bottleneck_features_train)
    
    test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    shuffle = False,
    target_size=(150,150),
    class_mode = None)
    bottleneck_features_test = base_model.predict_generator(
        test_generator, test_df.shape[0]//batch_size)
    np.save(open('../tuning_data/bottleneck_features_test.npy','wb'),
           bottleneck_features_test)

#### Save bottleneck_features

In [13]:
save_bottleneck_features(train_df,test_df,'eyewear',96)

Found 505 validated image filenames.
Found 127 validated image filenames.


In [21]:
train_data = np.load(open('../tuning_data/bottleneck_features_train.npy','rb'))
train_label = np.array(train_df.eyewear.map({'not_'+'eyewear':0, 'eyewear':1}))
len(train_label)%96

25

### Quick tuning of top models 

In [31]:
def train_top_model(train_df, test_df, epoch, batch_size, label):
    '''
    inputs:
    train_df, test_df: dataframes saved in pickle_files to generate train and test labels 
    epoch: num of epochs in fit 
    batch_size: same as image generator batch size 
    label: a string, eyewear, hat, or beard
    output:
    saves model weights in a folder 
    '''
    train_data = np.load(open('../tuning_data/bottleneck_features_train.npy','rb'))
    # make sure train_data and train_label have same num of samples
    train_label = np.array(train_df.eyewear.map({'not_'+label:0, label:1}))[:-(train_df.shape[0]%batch_size)]
    
    test_data = np.load(open('../tuning_data/bottleneck_features_test.npy','rb'))
    test_label = np.array(test_df.eyewear.map({'not_'+label:0, label:1}))[:-(test_df.shape[0]%batch_size)]
    
    # build top model
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    model.fit(train_data, train_label,
             epochs=epoch,
             batch_size=batch_size,
             validation_data=(test_data,test_label))
    model.save_weights('../tuning_data/bottleneck_vgg_model.h5')

#### run train_top_model and save results in tuning_data

In [32]:
train_top_model(train_df, test_df, 50, 96, 'eyewear')

Train on 480 samples, validate on 96 samples
Epoch 1/50
480/480 [==============================] - 0s 687us/step - loss: 3.3371 - acc: 0.5625 - val_loss: 2.2513 - val_acc: 0.5000
Epoch 2/50
480/480 [==============================] - 0s 37us/step - loss: 1.8445 - acc: 0.5750 - val_loss: 0.4074 - val_acc: 0.8750
Epoch 3/50
480/480 [==============================] - 0s 39us/step - loss: 0.5149 - acc: 0.7771 - val_loss: 0.5397 - val_acc: 0.6771
Epoch 4/50
480/480 [==============================] - 0s 42us/step - loss: 0.6530 - acc: 0.7458 - val_loss: 0.3292 - val_acc: 0.9167
Epoch 5/50
480/480 [==============================] - 0s 41us/step - loss: 0.3033 - acc: 0.8708 - val_loss: 0.4934 - val_acc: 0.7604
Epoch 6/50
480/480 [==============================] - 0s 42us/step - loss: 0.5840 - acc: 0.7333 - val_loss: 0.2958 - val_acc: 0.9167
Epoch 7/50
480/480 [==============================] - 0s 44us/step - loss: 0.1975 - acc: 0.9375 - val_loss: 0.3794 - val_acc: 0.8125
Epoch 8/50
480/480 [===

### Fine Tune Top Model to improve accuracy

In [41]:
def fine_tune_model(train_df, test_df,epoch, batch_size,label, print_model = True):
    # build VGG16 model and freeze top layers
    # input_shape: width, height, RGB (from image generator)
    model_vgg = VGG16(weights='imagenet',include_top=False, input_shape=(150,150,3))
    # build top model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
    top_model.add(Dense(256,activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))
    
    # load saved weights to fine tune parameters 
    top_model.load_weights('../tuning_data/bottleneck_vgg_model.h5')
    # add top model to model
    model = Model(inputs=model_vgg.input, outputs=top_model(model_vgg.output))
    # we will tune last 5 layers of the model: block5 and fully connected layer 
    for layer in model.layers[:15]:
        layer.trainable = False
    # we can tune the parameters for lr and momentum later to get better results
    model.compile(loss='binary_crossentropy',
             optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
             metrics=['accuracy'])
    # prepare train generator using data augmentation to battle small sample size 
    train_gen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    # not want to augment the test 
    test_gen = ImageDataGenerator(rescale=1./255)
    
    train_generator =  train_gen.flow_from_dataframe(
    dataframe=train_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    target_size=(150,150),
    class_mode = 'binary')
    
    test_generator =  test_gen.flow_from_dataframe(
    dataframe=test_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    target_size=(150,150),
    class_mode = 'binary')
    
    # run and fit model 
    history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_df.shape[0]//batch_size,
    epochs=epoch,
    validation_data=test_generator,
    validation_steps=test_df.shape[0]//batch_size,
    verbose=1)
    
    if print_model:
        model.summary()
    return history                  

In [42]:
model_history = fine_tune_model(train_df, test_df,50,96,'eyewear')

Found 505 validated image filenames belonging to 2 classes.
Found 127 validated image filenames belonging to 2 classes.
Epoch 1/50
5/5 [==============================] - 4s 736ms/step - loss: 6.6087 - acc: 0.0855 - val_loss: 4.0217 - val_acc: 0.1458
Epoch 2/50
5/5 [==============================] - 4s 743ms/step - loss: 2.4473 - acc: 0.2725 - val_loss: 1.5629 - val_acc: 0.3548
Epoch 3/50
5/5 [==============================] - 3s 561ms/step - loss: 1.0780 - acc: 0.4984 - val_loss: 0.8595 - val_acc: 0.5000
Epoch 4/50
5/5 [==============================] - 3s 609ms/step - loss: 0.8200 - acc: 0.4920 - val_loss: 0.7002 - val_acc: 0.6129
Epoch 5/50
5/5 [==============================] - 4s 796ms/step - loss: 0.7123 - acc: 0.5021 - val_loss: 0.6650 - val_acc: 0.5938
Epoch 6/50
5/5 [==============================] - 3s 663ms/step - loss: 0.7147 - acc: 0.5293 - val_loss: 0.6652 - val_acc: 0.5806
Epoch 7/50
5/5 [==============================] - 4s 740ms/step - loss: 0.6869 - acc: 0.5573 - val_l